In [59]:
import os;
import json;
import builtins
import pandas as pd;
import duckdb

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
threads = ["1"]
op = ['index']
datasets = ['fb', 'wiki', 'uniform_dense', 'uniform_sparse', 'normal', 'lognormal', 'osm', 'books']
test_results = []
index_stats = []
for op in op:
    for dataset in datasets:
        for thread in threads:
            testdir = os.path.join(dir, "_".join([op, dataset, thread]), "input_results")
            if (not os.path.exists(testdir)):
                continue
            rundirs = os.listdir(testdir)
            for rundir in rundirs:
                for test_result_file in os.listdir(os.path.join(testdir, rundir)):
                    json_file = builtins.open(os.path.join(testdir, rundir, test_result_file))
                    test_result = json.load(json_file)
                    print(test_result)
                    for t in test_result['result']['index_stats']:
                        t['dataset'] = dataset
                        t['op'] = op
                    index_stats += (test_result['result']['index_stats'])
                    if dataset == 'osm' or dataset == 'books':
                        test_result['input_size'] = 800_000_000 
                    else:
                        test_result['input_size'] = 200_000_000 
                    test_results.append(test_result)
                    json_file.close()
test_dataframe = pd.json_normalize(index_stats, max_level=2)
print(test_dataframe)
print(test_dataframe.columns)

{'command': 'numactl -N 1 -m 1 sponge/build/benchmark_runner sponge/index_fb_1/input_configs/inner', 'result': {'duration_sec': 86.22793579101562, 'index_stats': [{'index_build_duration': 14760980423, 'index_name': 'pgm256', 'index_size': 4143880}, {'index_build_duration': 13806518009, 'index_name': 'pgm1024', 'index_size': 810544}, {'index_build_duration': 13223902722, 'index_name': 'pgm2048', 'index_size': 296992}, {'index_build_duration': 2169372029, 'index_name': 'btree256', 'index_size': 12627696}, {'index_build_duration': 2109434645, 'index_name': 'btree1048', 'index_size': 3160008}, {'index_build_duration': 2105065225, 'index_name': 'btree2048', 'index_size': 1586176}], 'input_created': True}, 'spec': {'algo': 'create_input', 'key_size': 8, 'key_type': 'uint64', 'method': 'sosd', 'name': 'inner', 'num_keys': 200000000, 'result_path': 'sponge/index_fb_1/inputs/inner', 'source': '/home/chesetti/sosd-data/fb_200M_uint64', 'value_size': 8, 'write_result_to_disk': True}}
{'command': 

In [76]:
epsilon = {
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "btree256": 256,
    "btree512": 512,
    "btree1024": 1024,
    "btree1048": 1024,
    "btree2048": 2048,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
indexType = {
    "pgm64": "PGM",
    "pgm128": "PGM",
    "pgm256": "PGM",
    "pgm512": "PGM",
    "pgm1024": "PGM",
    "pgm2048": "PGM",
    "btree256": "BTREE",
    "btree512": "BTREE",
    "btree1024": "BTREE",
    "btree1048": "BTREE",
    "btree2048": "BTREE",
    "hashjoin": "NA",
    "btree": 0,
    "sj": "SJ",
    "sj2": "NA",
    "standard_merge": "SM"
};
dataset_type = {
    "fb": "REAL",
    "wiki": "REAL",
    "osm": "REAL",
    "books": "REAL",
    "uniform_dense": "SYNTHETIC",
    "uniform_sparse": "SYNTHETIC",
    "normal": "SYNTHETIC",
    "lognormal": "SYNTHETIC",
};
dataset_display = {
    "fb": "fb",
    "wiki": "wiki",
    "osm": "osm",
    "books": "books",
    "uniform_dense": "udense",
    "uniform_sparse": "usparse",
    "normal": "normal",
    "lognormal": "lognormal",
};
dataset_order = {
    "fb": 0,
    "wiki": 1,
    "osm": 2,
    "books": 3,
    "uniform_dense": 4,
    "uniform_sparse": 5,
    "normal": 6,
    "lognormal": 7,
};
test_dataframe["epsilon"] = test_dataframe["index_name"].map(lambda x: epsilon[x.lower()])
test_dataframe["index_type"] = test_dataframe["index_name"].map(lambda x: indexType[x.lower()])
test_dataframe["dataset_type"] = test_dataframe["dataset"].map(lambda x: dataset_type[x.lower()])
test_dataframe["dataset_display"] = test_dataframe["dataset"].map(lambda x: dataset_display[x.lower()])
test_dataframe["dataset_order"] = test_dataframe["dataset"].map(lambda x: dataset_order[x.lower()])
test_dataframe["index_size_MB"] = test_dataframe["index_size"].map(lambda x: x / (1024 * 1024))
test_dataframe["index_build_duration_sec"] = test_dataframe["index_build_duration"].map(lambda x: x / (1e9))


index_size = test_dataframe.pivot(index='epsilon', values="index_size_MB", columns=["dataset_order", "dataset_display", "index_type"])
index_duration = test_dataframe.pivot(index='epsilon', values="index_build_duration_sec", columns=["dataset_order", "dataset_display", "index_type"])
columns = list(index_size.columns)
columns.sort()
index_size[columns]

print(index_size[columns].to_latex(float_format='%.3f'))
print(index_duration[columns].to_latex(float_format='%.2f'))

\begin{tabular}{lrrrrrrrrrrrrrrrr}
\toprule
dataset_order & \multicolumn{2}{r}{0} & \multicolumn{2}{r}{1} & \multicolumn{2}{r}{4} & \multicolumn{2}{r}{5} & \multicolumn{2}{r}{6} & \multicolumn{2}{r}{7} & \multicolumn{2}{r}{2} & \multicolumn{2}{r}{3} \\
dataset_display & \multicolumn{2}{r}{fb} & \multicolumn{2}{r}{wiki} & \multicolumn{2}{r}{udense} & \multicolumn{2}{r}{usparse} & \multicolumn{2}{r}{normal} & \multicolumn{2}{r}{lognormal} & \multicolumn{2}{r}{osm} & \multicolumn{2}{r}{books} \\
index_type & PGM & BTREE & PGM & BTREE & PGM & BTREE & PGM & BTREE & PGM & BTREE & PGM & BTREE & PGM & BTREE & PGM & BTREE \\
epsilon &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
256 & 3.952 & 12.043 & 0.295 & 12.043 & 0.000 & 12.043 & 0.053 & 12.043 & 0.011 & 12.043 & 0.015 & 12.043 & 10.484 & 48.159 & 4.101 & 48.159 \\
1024 & 0.773 & 3.014 & 0.106 & 3.014 & 0.000 & 3.014 & 0.003 & 3.014 & 0.005 & 3.014 & 0.008 & 3.014 & 2.565 & 12.043 & 0.348 & 12.043 \\
2048 & 0.283 & 1.513 & 0.0